# Unit Testing `GiRaFFE_NRPy`: $A_k$ to $B^i$

### Author: Patrick Nelson

This notebook validates our A-to-B solver for use in `GiRaFFE_NRPy`. Because the original `GiRaFFE` used staggered grids and we do not, we can not trivially do a direct comparison to the old code. Instead, we will compare the numerical results with the expected analytic results. 

**Module Status:** <font color=red><b> In-Progress </b></font>

**Validation Notes:** This module will validate the routines in [Tutorial-GiRaFFE_HO_C_code_library-A2B](../Tutorial-GiRaFFE_HO_C_code_library-A2B.ipynb).

It is, in general, good coding practice to unit test functions individually to verify that they produce the expected and intended output. Here, we expect our functions to produce the correct cross product in an arbitrary spacetime. To that end, we will choose functions that are easy to differentiate, but lack the symmetries that would trivialize the finite-difference algorithm. Higher-order polynomials are one such type of function. 

We will start with the simplest case - testing the second-order solver. In second-order finite-differencing, we use a three-point stencil that can exactly differentiate polynomials up to quadratic. So, we will use cubic functions three variables. For instance,

\begin{align}
A_x &= ax^3 + by^3 + cz^3 + dy^2 + ez^2 + f \\
A_y &= gx^3 + hy^3 + lz^3 + mx^2 + nz^2 + p \\
A_z &= px^3 + qy^3 + rz^3 + sx^2 + ty^2 + u. \\
\end{align}

It will be much simpler to let NRPy+ handle most of this work. So, we will import the core functionality of NRPy+, build the expressions, and then output them using `outputC()`.

In [1]:
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions
# First, we'll add the parent directory to the list of directories Python will check for modules.
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import *            # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

out_dir = "Validation/"
cmd.mkdir(out_dir)

thismodule = "Unit_Test_GiRaFFE_NRPy_Ccode_library_A2B"
a,b,c,d,e,f,g,h,l,m,n,o,p,q,r,s,t,u = par.Cparameters("REAL",thismodule,["a","b","c","d","e","f","g","h","l","m","n","o","p","q","r","s","t","u"],10.0)
gammadet = gri.register_gridfunctions("AUXEVOL","gammadet")

DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()
x = rfm.xxCart[0]
y = rfm.xxCart[1]
z = rfm.xxCart[2]

AD = ixp.register_gridfunctions_for_single_rank1("EVOL","AD")
AD[0] = a*x**3 + b*y**3 + c*z**3 + d*y**2 + e*z**2 + f
AD[1] = g*x**3 + h*y**3 + l*z**3 + m*x**2 + n*z**2 + o
AD[2] = p*x**3 + q*y**3 + r*z**3 + s*x**2 + t*y**2 + u


Next, we'll let NRPy+ compute derivatives analytically according to $$B^i = \frac{[ijk]}{\sqrt{\gamma}} \partial_j A_k.$$ Then we can carry out two separate tests to verify the numerical derivatives. First, we will verify that when we let the cubic terms be zero, the two calculations of $B^i$ agree to roundoff error. Second, we will verify that when we set the cubic terms, our error is dominated by trunction error that converges to zero at the expected rate. 

In [2]:
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaUUU = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LeviCivitaUUU[i][j][k] = LeviCivitaDDD[i][j][k] / sp.sqrt(gammadet)
            
B_analyticU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_analyticU")
for i in range(DIM):
    B_analyticU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            B_analyticU[i] += LeviCivitaUUU[i][j][k] * sp.diff(AD[k],rfm.xxCart[j])


Now that we have our vector potential and analytic magnetic field to compare against, we will start writing our unit test. We'll also import common C functionality, define `REAL`, the number of ghost zones, and the faces, and set the standard macros for NRPy+ style memory access.

In [3]:
out_string = """
// These are common packages that we are likely to need.
#include "stdio.h"
#include "stdlib.h"
#include "math.h"
#include "string.h" // Needed for strncmp, etc.
#include "stdint.h" // Needed for Windows GCC 6.x compatibility
#include <time.h>   // Needed to set a random seed.

# define REAL double
const int MAXFACE = -1;
const int NUL     = +0;
const int MINFACE = +1;
const int NGHOSTS = 1;

// Standard NRPy+ memory access:
#define IDX4(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * ( (k) + Nxx_plus_2NGHOSTS[2] * (g) ) ) )
#define IDX3(i,j,k) ( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * (k) ) )
// Assuming idx = IDX3(i,j,k). Much faster if idx can be reused over and over:
#define IDX4pt(g,idx)   ( (idx) + (Nxx_plus_2NGHOSTS[0]*Nxx_plus_2NGHOSTS[1]*Nxx_plus_2NGHOSTS[2]) * (g) )

"""

We'll now define the gridfunction names.

In [4]:
out_string += """
// Let's also #define the NRPy+ gridfunctions
#define AD0GF 0
#define AD1GF 1
#define AD2GF 2
#define NUM_EVOL_GFS 3

#define GAMMADETGF 0
#define B_ANALYTICU0GF 1
#define B_ANALYTICU1GF 2
#define B_ANALYTICU2GF 3
#define BU0GF 4
#define BU1GF 5
#define BU2GF 6
#define NUM_AUXEVOL_GFS 7

"""

Now, we'll handle the different A2B codes. There are several things to do here. First, we'll add `#include`s to the C code so that we have access to the functions we want to test. We must also create a directory and copy the files to that directory. We will choose to do this in the subfolder `A2B` relative to this tutorial.

In [5]:
out_string += """
#include "../A2B/driver_AtoB.c" // This file contains both functions we need.

"""

cmd.mkdir(os.path.join("A2B/"))
shutil.copy(os.path.join("../GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/driver_AtoB.c"),os.path.join("A2B/"))


We also should write a function that will use the analytic formulae for $B^i$. Then, we'll need to call the function from the module `GiRaFFE_HO_A2B` to generate the different header files. Also, we will declare the parameters for the vector potential functions.

In [6]:
out_string += """
REAL a,b,c,d,e,f,g,h,l,m,n,o,p,q,r,s,t,u;

void calculate_exact_BU(const int Nxx_plus_2NGHOSTS[3],REAL *xx[3],double *auxevol_gfs) {
    for(int i2=0;i2<Nxx_plus_2NGHOSTS[2];i2++) for(int i1=0;i1<Nxx_plus_2NGHOSTS[1];i1++) for(int i0=0;i0<Nxx_plus_2NGHOSTS[0];i0++) {
        REAL xx0 = xx[0][i0];
        REAL xx1 = xx[1][i1];
        REAL xx2 = xx[2][i2];
"""

B_analyticU_to_print   = [\
                           lhrh(lhs=gri.gfaccess("out_gfs","B_analyticU0"),rhs=B_analyticU[0]),\
                           lhrh(lhs=gri.gfaccess("out_gfs","B_analyticU1"),rhs=B_analyticU[1]),\
                           lhrh(lhs=gri.gfaccess("out_gfs","B_analyticU2"),rhs=B_analyticU[2]),\
                          ]
B_analyticU_kernel = fin.FD_outputC("returnstring",B_analyticU_to_print,params="outCverbose=False")
out_string += B_analyticU_kernel

out_string += """        
    }
}

"""

gri.glb_gridfcs_list = []
import GiRaFFE_HO.GiRaFFE_HO_A2B as A2B
# We'll generate these into the A2B subdirectory since that's where the functions
# we're testing expect them to be.
A2B.GiRaFFE_HO_A2B("A2B/")


Wrote to file "A2B/B_from_A_order10.h"
Wrote to file "A2B/B_from_A_order8.h"
Wrote to file "A2B/B_from_A_order6.h"
Wrote to file "A2B/B_from_A_order4.h"
Wrote to file "A2B/B_from_A_order2.h"
Wrote to file "A2B/B_from_A_order2_dirx0_dnwind.h"
Wrote to file "A2B/B_from_A_order2_dirx0_upwind.h"
Wrote to file "A2B/B_from_A_order2_dirx1_dnwind.h"
Wrote to file "A2B/B_from_A_order2_dirx1_upwind.h"
Wrote to file "A2B/B_from_A_order2_dirx2_dnwind.h"
Wrote to file "A2B/B_from_A_order2_dirx2_upwind.h"


We'll now write a function to set the vector potential $A_k$. This simply uses NRPy+ to generte most of the code from the expressions we wrote at the beginning. 

In [7]:
out_string += """
void calculate_AD(const int Nxx_plus_2NGHOSTS[3],REAL *xx[3],double *out_gfs) {
    for(int i2=0;i2<Nxx_plus_2NGHOSTS[2];i2++) for(int i1=0;i1<Nxx_plus_2NGHOSTS[1];i1++) for(int i0=0;i0<Nxx_plus_2NGHOSTS[0];i0++) {
        REAL xx0 = xx[0][i0];
        REAL xx1 = xx[1][i1];
        REAL xx2 = xx[2][i2];
"""

AD_to_print = [\
               lhrh(lhs=gri.gfaccess("out_gfs","AD0"),rhs=AD[0]),\
               lhrh(lhs=gri.gfaccess("out_gfs","AD1"),rhs=AD[1]),\
               lhrh(lhs=gri.gfaccess("out_gfs","AD2"),rhs=AD[2]),\
              ]
AD_kernel = fin.FD_outputC("returnstring",AD_to_print,params="outCverbose=False")
out_string += AD_kernel

out_string += """        
    }
}
"""

We will define the extent of our grid here. 

In [8]:
out_string += """
const REAL xmin = -0.01,xmax=0.01;
const REAL ymin = -0.01,ymax=0.01;
const REAL zmin = -0.01,zmax=0.01;

"""

Now, we'll write the main method. First, we'll set up the grid. In this test, we cannot use only one point. As we are testing a three-point stencil, we can get away with a minimal $3 \times 3 \times 3$ grid. Then, we'll write the A fields. After that, we'll calculate the magnetic field two ways.

In [9]:
out_string += """
main() {
    // For the first trial, we'll use this grid. It has one point and one ghost zone.
    const int Nxx[3] = {1,1,1};
    const int Nxx_plus_2NGHOSTS[3] = {3,3,3};
    
    const REAL xxmin[3] = {xmin,ymin,zmin};
    const REAL xxmax[3] = {xmax,ymax,zmax};

    REAL dxx[3];
    for(int i=0;i<3;i++) dxx[i] = (xxmax[i] - xxmin[i]) / ((REAL)Nxx[i]+1.0);
    for(int i=0;i<3;i++) printf("dxx[%d] = %.15e\\n",i,dxx[i]);
    
    // We'll define our grid slightly different from how we normally would. We let our outermost
    // ghostzones coincide with xxmin and xxmax instead of the interior of the grid. This means
    // that the ghostzone points will have identical positions so we can do convergence tests of them.

    // Step 0d.ii: Set up uniform coordinate grids
    REAL *xx[3];
    for(int i=0;i<3;i++) {
        xx[i] = (REAL *)malloc(sizeof(REAL)*Nxx_plus_2NGHOSTS[i]);
        for(int j=0;j<Nxx_plus_2NGHOSTS[i];j++) {
            xx[i][j] = xxmin[i] + ((REAL)(j))*dxx[i]; // Face-centered grid.
        }
    }
    //for(int i=0;i<Nxx_plus_2NGHOSTS[0];i++) printf("xx[0][%d] = %.15e\\n",i,xx[0][i]);
    
    // This is the array to which we'll write the NRPy+ variables.
    REAL *auxevol_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_AUXEVOL_GFS * Nxx_plus_2NGHOSTS[2] * Nxx_plus_2NGHOSTS[1] * Nxx_plus_2NGHOSTS[0]);
    REAL *evol_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS[2] * Nxx_plus_2NGHOSTS[1] * Nxx_plus_2NGHOSTS[0]);
    
    for(int i2=0;i2<Nxx_plus_2NGHOSTS[2];i2++) for(int i1=0;i1<Nxx_plus_2NGHOSTS[1];i1++) for(int i0=0;i0<Nxx_plus_2NGHOSTS[0];i0++) {
        auxevol_gfs[IDX4(GAMMADETGF,i0,i1,i2)] = 1.0;
    }
    
    // We now want to set up the vector potential. First, we must set the coefficients. 
    // We will use random integers between -10 and 10. For the first test, we let the 
    // Cubic coefficients remain zero. Those are a,b,c,g,h,l,p,q, and r.
    d = (double)(rand()%20-10);
    e = (double)(rand()%20-10);
    f = (double)(rand()%20-10);
    m = (double)(rand()%20-10);
    n = (double)(rand()%20-10);
    o = (double)(rand()%20-10);
    s = (double)(rand()%20-10);
    t = (double)(rand()%20-10);
    u = (double)(rand()%20-10);

    calculate_AD(Nxx_plus_2NGHOSTS,xx,evol_gfs);

    // We'll also calculate the exact solution for B^i
    calculate_exact_BU(Nxx_plus_2NGHOSTS,xx,auxevol_gfs);
    
    // And now for the numerical derivatives:
    driver_A_to_B(Nxx,Nxx_plus_2NGHOSTS,dxx,evol_gfs,auxevol_gfs);
    
    //Two variables for inside the loop:
    int ghost_zone_overlap;int indices[3];
    for(int i2=0;i2<Nxx_plus_2NGHOSTS[2];i2++) for(int i1=0;i1<Nxx_plus_2NGHOSTS[1];i1++) for(int i0=0;i0<Nxx_plus_2NGHOSTS[0];i0++) {
        // Are we on an edge/vertex? This algorithm can probably be improved.
        ghost_zone_overlap = 0;
        indices[0] = i0;
        indices[1] = i1;
        indices[2] = i2;
        for(int dim=0;dim<3;dim++) {
            if(indices[dim]%(Nxx[dim]+NGHOSTS)<NGHOSTS) {
                ghost_zone_overlap++;
            }
        }
        if (ghost_zone_overlap < 2) {
            // Don't print if we're on an edge or vertex
            printf("Analytic: %.16e, %.16e, %.16e\\n",auxevol_gfs[IDX4(B_ANALYTICU0GF,i0,i1,i2)],auxevol_gfs[IDX4(B_ANALYTICU1GF,i0,i1,i2)],auxevol_gfs[IDX4(B_ANALYTICU2GF,i0,i1,i2)]);
            printf("Numeric:  %.16e, %.16e, %.16e\\n\\n",auxevol_gfs[IDX4(BU0GF,i0,i1,i2)],auxevol_gfs[IDX4(BU1GF,i0,i1,i2)],auxevol_gfs[IDX4(BU2GF,i0,i1,i2)]);
        }
    }
    
    // Now, we'll set the cubic coefficients:
    a = (double)(rand()%20-10);
    b = (double)(rand()%20-10);
    c = (double)(rand()%20-10);
    g = (double)(rand()%20-10);
    h = (double)(rand()%20-10);
    l = (double)(rand()%20-10);
    p = (double)(rand()%20-10);
    q = (double)(rand()%20-10);
    r = (double)(rand()%20-10);
    
    // And recalculate on our initial grid:
    calculate_AD(Nxx_plus_2NGHOSTS,xx,evol_gfs);

    // We'll also calculate the exact solution for B^i
    calculate_exact_BU(Nxx_plus_2NGHOSTS,xx,auxevol_gfs);
    
    // And now for the numerical derivatives:
    driver_A_to_B(Nxx,Nxx_plus_2NGHOSTS,dxx,evol_gfs,auxevol_gfs);
    
    // To do a convergence test, we'll also need a second grid with twice the resolution.
    // We'll need new Nxx and Nxx_plus_2NGHOSTS terms; we'll use the suffix *_o2 to represent the halved grid spacing.
    const int Nxx_o2[3] = {3,3,3};
    const int Nxx_plus_2NGHOSTS_o2[3] = {5,5,5};
    // We set up these grids in a bit of an unusual way so that the points in the ghost zones line up for testing.

    REAL dxx_o2[3];
    for(int i=0;i<3;i++) dxx_o2[i] = (xxmax[i] - xxmin[i]) / ((REAL)Nxx_o2[i]+1.0);
    for(int i=0;i<3;i++) printf("dxx_o2[%d] = %.15e\\n",i,dxx_o2[i]);

    REAL *xx_o2[3];
    for(int i=0;i<3;i++) {
        xx_o2[i] = (REAL *)malloc(sizeof(REAL)*Nxx_plus_2NGHOSTS_o2[i]);
        for(int j=0;j<Nxx_plus_2NGHOSTS_o2[i];j++) {
            xx_o2[i][j] = xxmin[i] + ((REAL)(j))*dxx_o2[i]; // Face-centered grid.
        }
    }
    //for(int i=0;i<Nxx_plus_2NGHOSTS_o2[0];i++) printf("xx_o2[0][%d] = %.15e\\n",i,xx_o2[0][i]);

    // This is the array to which we'll write the NRPy+ variables.
    REAL *auxevol_o2_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_AUXEVOL_GFS * Nxx_plus_2NGHOSTS_o2[2] * Nxx_plus_2NGHOSTS_o2[1] * Nxx_plus_2NGHOSTS_o2[0]);
    REAL *evol_o2_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_o2[2] * Nxx_plus_2NGHOSTS_o2[1] * Nxx_plus_2NGHOSTS_o2[0]);
    
    for(int i2=0;i2<Nxx_plus_2NGHOSTS_o2[2];i2++) for(int i1=0;i1<Nxx_plus_2NGHOSTS_o2[1];i1++) for(int i0=0;i0<Nxx_plus_2NGHOSTS_o2[0];i0++) {
        auxevol_o2_gfs[IDX4(GAMMADETGF,i0,i1,i2)] = 1.0;
    }
    
    // Now, we'll calculate the fields on the refined grid.
    calculate_AD(Nxx_plus_2NGHOSTS_o2,xx_o2,evol_o2_gfs);

    // We'll also calculate the exact solution for B^i
    calculate_exact_BU(Nxx_plus_2NGHOSTS_o2,xx_o2,auxevol_o2_gfs);
    
    // And now for the numerical derivatives:
    driver_A_to_B(Nxx_o2,Nxx_plus_2NGHOSTS_o2,dxx_o2,evol_o2_gfs,auxevol_o2_gfs);
    
    for(int i2=0;i2<Nxx_plus_2NGHOSTS_o2[2];i2++) for(int i1=0;i1<Nxx_plus_2NGHOSTS_o2[1];i1++) for(int i0=0;i0<Nxx_plus_2NGHOSTS_o2[0];i0++) {
        // Are we on an edge/vertex? This algorithm can probably be improved.
        ghost_zone_overlap = 0;
        indices[0] = i0;
        indices[1] = i1;
        indices[2] = i2;
        for(int dim=0;dim<3;dim++) {
            if(indices[dim]%(Nxx_o2[dim]+NGHOSTS)<NGHOSTS) {
                ghost_zone_overlap++;
            }
        }
        if (ghost_zone_overlap < 2) {
            // Don't print if we're on an edge or vertex
            /*printf("i0,i1,i2 = %d,%d,%d\\n",i0,i1,i2);
            printf("Analytic: %.16e, %.16e, %.16e\\n",auxevol_o2_gfs[IDX4(B_ANALYTICU0GF,i0,i1,i2)],auxevol_o2_gfs[IDX4(B_ANALYTICU1GF,i0,i1,i2)],auxevol_o2_gfs[IDX4(B_ANALYTICU2GF,i0,i1,i2)]);
            printf("Numeric:  %.16e, %.16e, %.16e\\n\\n",auxevol_o2_gfs[IDX4(BU0GF,i0,i1,i2)],auxevol_o2_gfs[IDX4(BU1GF,i0,i1,i2)],auxevol_o2_gfs[IDX4(BU2GF,i0,i1,i2)]);*/
        }
    }
    
    for(int i2=0;i2<Nxx_plus_2NGHOSTS[2];i2++) for(int i1=0;i1<Nxx_plus_2NGHOSTS[1];i1++) for(int i0=0;i0<Nxx_plus_2NGHOSTS[0];i0++) {
        // Are we on an edge/vertex? This algorithm can probably be improved.
        ghost_zone_overlap = 0;
        indices[0] = i0;
        indices[1] = i1;
        indices[2] = i2;
        for(int dim=0;dim<3;dim++) {
            if(indices[dim]%(Nxx[dim]+NGHOSTS)<NGHOSTS) {
                ghost_zone_overlap++;
            }
        }
        if (ghost_zone_overlap < 2) {
            // Don't print if we're on an edge or vertex 
            /*printf("Convergence: %.16e, %.16e, %.16e\\n",
                   log(fabs((auxevol_gfs[IDX4(B_ANALYTICU0GF,i0,i1,i2)]-auxevol_gfs[IDX4(BU0GF,i0,i1,i2)])/(auxevol_gfs[IDX4(B_ANALYTICU0GF,i0,i1,i2)]-auxevol_o2_gfs[IDX4(B_ANALYTICU0GF,2*i0,2*i1,2*i2)])))/log(2),
                   log(fabs((auxevol_gfs[IDX4(B_ANALYTICU1GF,i0,i1,i2)]-auxevol_gfs[IDX4(BU1GF,i0,i1,i2)])/(auxevol_gfs[IDX4(B_ANALYTICU1GF,i0,i1,i2)]-auxevol_o2_gfs[IDX4(B_ANALYTICU1GF,2*i0,2*i1,2*i2)])))/log(2),
                   log(fabs((auxevol_gfs[IDX4(B_ANALYTICU2GF,i0,i1,i2)]-auxevol_gfs[IDX4(BU2GF,i0,i1,i2)])/(auxevol_gfs[IDX4(B_ANALYTICU2GF,i0,i1,i2)]-auxevol_o2_gfs[IDX4(B_ANALYTICU2GF,2*i0,2*i1,2*i2)])))/log(2)
                   );*/
        }
    }

}
"""


Now, we must write out the code to a `.C` file.

In [10]:
with open(os.path.join(out_dir,"A2B_unit_test.C"),"w") as file:
    file.write(out_string)

Now that we have our file, we can compile it and run the executable.

In [11]:
import time

print("Now compiling, should take ~2 seconds...\n")
start = time.time()
cmd.C_compile(os.path.join(out_dir,"A2B_unit_test.C"), os.path.join(out_dir,"A2B_unit_test"))
end = time.time()
print("Finished in "+str(end-start)+" seconds.\n\n")

# os.chdir(out_dir)
print("Now running...\n")
start = time.time()
# cmd.Execute(os.path.join("Stilde_flux_unit_test"))
!./Validation/A2B_unit_test
end = time.time()
print("Finished in "+str(end-start)+" seconds.\n\n")
# os.chdir(os.path.join("../"))


Now compiling, should take ~2 seconds...

Compiling executable...
Executing `gcc -Ofast -fopenmp -march=native -funroll-loops Validation/A2B_unit_test.C -o Validation/A2B_unit_test -lm`...
Finished executing in 1.22334718704 seconds.
Finished compilation.
Finished in 1.23518800735 seconds.


Now running...

dxx[0] = 1.000000000000000e-02
dxx[1] = 1.000000000000000e-02
dxx[2] = 1.000000000000000e-02
Analytic: 5.9999999999999998e-02, 8.0000000000000002e-02, 0.0000000000000000e+00
Numeric:  6.0000000000037801e-02, 7.9999999999991189e-02, 0.0000000000000000e+00

Analytic: -4.0000000000000001e-02, 0.0000000000000000e+00, -1.4000000000000001e-01
Numeric:  -3.9999999999995595e-02, 0.0000000000000000e+00, -1.4000000000002899e-01

Analytic: 0.0000000000000000e+00, -8.0000000000000002e-02, -1.0000000000000001e-01
Numeric:  0.0000000000000000e+00, -7.9999999999991189e-02, -9.9999999999944578e-02

Analytic: 0.0000000000000000e+00, 0.0000000000000000e+00, 0.0000000000000000e+00
Numeric:  0.00000000